# make more variables to analyze

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

In [19]:
## Read in data and fix some base cases, save the good stuff
proposed_gen_data = pd.read_csv('./Clean Data/proposed_gen_master_list.csv')
proposed_gen_data = proposed_gen_data.drop(columns='Unnamed: 0')
proposed_gen_data = proposed_gen_data[~proposed_gen_data.utility_id.str.contains('NOTE', na=False)]
proposed_gen_data.to_csv('./Clean Data/proposed_gen_master_list.csv')

AttributeError: Can only use .str accessor with string values!

In [24]:
proposed_gen_data = pd.read_csv('./Clean Data/proposed_gen_master_list.csv')
proposed_gen_data = proposed_gen_data.drop(columns='Unnamed: 0')

In [26]:
proposed_gen_data['unique_id'] = proposed_gen_data.groupby(['utility_name']).grouper.group_info[0]
proposed_gen_data

,bypass_heat_recovery,carbon_capture,chp,cofire_fuels,cogen,curr_sceheduled_month,curr_scheduled_year,distributed_gen,duct_burners,energy_source,...,technology,transportation,ultrasupercritical,unit_code,utility_id,utility_name,winter_cap_eia,winter_cap_resp,year,unique_id
0,NaN,NaN,NaN,NaN,N,NaN,NaN,N,NaN,"['NG', nan, nan, nan, nan, nan]",...,NaN,"['PL', nan, nan, nan, nan, nan]",NaN,NaN,20,AES Cypress LLC,187.38,175.0,2004,45
1,NaN,NaN,NaN,NaN,N,NaN,NaN,N,NaN,"['NG', nan, nan, nan, nan, nan]",...,NaN,"['PL', nan, nan, nan, nan, nan]",NaN,NaN,20,AES Cypress LLC,187.38,175.0,2004,45
2,NaN,NaN,NaN,NaN,N,NaN,NaN,N,NaN,"['NG', nan, nan, nan, nan, nan]",...,NaN,"['PL', nan, nan, nan, nan, nan]",NaN,NaN,20,AES Cypress LLC,187.38,175.0,2004,45
3,NaN,NaN,NaN,NaN,N,NaN,NaN,N,NaN,"['NG', nan, nan, nan, nan, nan]",...,NaN,"['PL', nan, nan, nan, nan, nan]",NaN,NaN,46,AES Hoytdale LLC,169.20,177.0,2004,50
4,NaN,NaN,NaN,NaN,N,NaN,NaN,N,NaN,"['NG', nan, nan, nan, nan, nan]",...,NaN,"['PL', nan, nan, nan, nan, nan]",NaN,NaN,46,AES Hoytdale LLC,169.20,177.0,2004,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22190,X,NaN,N,NaN,NaN,11.0,2020.0,NaN,X,"['SUN', nan, nan, nan, nan, nan]",...,Solar Photovoltaic,[],NaN,NaN,49893,Invenergy Services LLC,NaN,190.0,2018,1511
22191,X,NaN,N,NaN,NaN,10.0,2020.0,NaN,X,"['WND', nan, nan, nan, nan, nan]",...,Onshore Wind Turbine,[],NaN,NaN,62758,"Orchard Windfarm, LLC",NaN,40.0,2018,2163
22192,X,NaN,N,NaN,NaN,12.0,2020.0,NaN,X,"['WND', nan, nan, nan, nan, nan]",...,Onshore Wind Turbine,[],NaN,NaN,62778,"Rattlesnake Flat, LLC",NaN,144.0,2018,2433
22193,X,NaN,N,NaN,NaN,12.0,2019.0,NaN,X,"['WND', nan, nan, nan, nan, nan]",...,Onshore Wind Turbine,[],NaN,NaN,59496,Allete Clean Energy,NaN,106.7,2018,128


## How many entries appear over multiple years?  i.e. How many unique entries do I actually have? Raw number is 22,195 entries, but some seem to be part of same plant...

In [297]:
## Goal is to create a unique ID for each unit in the sample because I am worried that some units will appear in multiple data sets. 

## Solution is to start with creating group_IDs for entries that have the same (plant code, utility name, nameplate cap, generator id) combinations

proposed_gen_data['temp_group_id'] = proposed_gen_data.fillna(method='ffill').groupby(['plant_code', 'utility_name','nameplate_cap', 'generator_id']).grouper.group_info[0]

#len(proposed_gen_data['temp_group_id'].unique())
# There are 10,380 unique temp_group_ids, so now I need to figure out why there are repeated codes

In [298]:
# I do this by selecting observations that are coded to the same group_ids

special_ids = proposed_gen_data.groupby(['temp_group_id']).size() > 1
special_ids = [i for i in special_ids.index if special_ids[i]]
special_ids = proposed_gen_data[proposed_gen_data['temp_group_id'].isin(special_ids)]

# special_ids.shape[0]
# There are 16843 entries with non-unique group_ids.

In [23]:
# Now I look to find out how many of the repeated entries are appearing in multiple years
df = special_ids.groupby('temp_group_id').status.nunique()
df
# For example this thing appears in multiple years... but it's cancelled
# special_ids[special_ids['temp_group_id']==1957].filter(['utility_name','plant_name','status'])

NameError: name 'special_ids' is not defined

In [313]:
status_codes = {}
#IP: Planned new generator cancelled, indefinitely postponed, or no longer in resource plan
status_codes['cancelled'] = 'IP'
#TS: Construction complete, but not yet in commercial operation
status_codes['complete_not_operating'] = 'TS'
#P: Planned for installation but regulatory approvals not initiated; not under construction
status_codes['planned_not_started'] = 'P'
#L: Regulatory approvals pending; not under construction but site preparation could be underway
status_codes['reg_approval_pending'] = 'L'
#T: Regulatory approvals received; but not under construction but site preparation could be underway
status_codes['reg_approval_received'] = 'T'
#U: Under construction, less than or equal to 50 percent complete (based on construction time to date of operation)
status_codes['under_construction'] = 'U' 
#V: Under construction, more than 50 percent complete (based on construction time to date of operation)
status_codes['almost_done'] = 'V'
# Other (described in Comments)
status_codes['other'] = 'OT'


In [312]:
special_ids['status'].value_counts()

IP    6011
P     3412
L     2063
U     1810
V     1424
T     1372
TS     721
OT      30
Name: status, dtype: int64

In [204]:
multi_yr_groups[multi_yr_groups['temp_group_id'] == 1091]

,bypass_heat_recovery,carbon_capture,chp,cofire_fuels,cogen,curr_sceheduled_month,curr_scheduled_year,distributed_gen,duct_burners,energy_source,...,technology,transportation,ultrasupercritical,unit_code,utility_id,utility_name,winter_cap_eia,winter_cap_resp,year,temp_group_id
7700,NaN,NaN,NaN,NaN,N,7.0,2010.0,NaN,NaN,"['NG', nan, nan, nan, nan, nan]",...,NaN,[],NaN,NaN,1015,Austin Energy,50.4,47.3,2008,1091
9822,NaN,NaN,NaN,NaN,N,7.0,2010.0,NaN,NaN,"['NG', nan, nan, nan, nan, nan]",...,NaN,[],NaN,NaN,1015,Austin Energy,50.4,47.3,2009,1091
